In [1]:
import pandas as pd 
from os.path import join
import joblib
import numpy as np

## Compute calibrated baseline predictions. 

In [2]:
baseline_cols = { 'tornado'     : 'uh_probs_>180_prob_max',
                  'severe_hail' : 'hail_probs_>1.0_prob_max',
                  'severe_wind' : 'wnd_probs_>40_prob_max',
                }

In [13]:
path = '/work/mflora/ML_DATA/DATA'

lead_time = 'second_hour'
year = '2021'
df = pd.read_pickle(join(path, f'ml_dataset_{lead_time}_{year}_reverted_names.pkl'))


In [14]:
np.unique(df['FCST_TIME_IDX'])

array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [3]:
# test_ml_dataset_first_hour_tornado.feather
path = '/work/mflora/ML_DATA/DATA'
new_path = '/work/mflora/ML_DATA/test_data'
#lead_time = 'first_hour'
#target = 'tornado'

for lead_time in ['first_hour', 'second_hour']:
    for target in ['tornado', 'severe_hail', 'severe_wind']:
        df = pd.read_pickle(join(path, f'{lead_time}_testing_matched_to_{target}_0km_dataset.pkl'))
        # Convert all but the run time & date to float type.
        objs = ['Run Time', 'Run Date']
        dtypes = {c : float for c in df.columns if c not in objs}
        dtypes['Run Date'] = int
        df = df.astype(dtypes)
        
        # load the calibration models
        model_path = '/work/mflora/ML_DATA/MODEL_SAVES'
        model = joblib.load(join(model_path,
                                 f'calibration_model_wofs_{lead_time}_{target}_{baseline_cols[target]}.joblib'))
        predictions = model.predict(df[baseline_cols[target]].values)
        df[f'Baseline__{target}'] = predictions 

        to_drop = [c for c in df.columns if 'prob_max' in c]
        to_drop.append('label')
        
        dtypes  = {k : i for k,i in dtypes.items() if k not in to_drop}
        
        test_df_list = [pd.read_pickle(join(path, 
                        f'ml_dataset_{lead_time}_{year}_reverted_names.pkl')).astype(dtypes,errors='ignore')
           for year in ['2020', '2021']]
            
        test_df_list.append(df)
        test_df = pd.concat(test_df_list, axis=0)
        test_df.reset_index(inplace=True, drop=True)
        
        columns_to_keep = [c for c in test_df.columns if c not in to_drop]
        
        test_df = test_df[columns_to_keep]
        
        out_file = join(new_path, f'test_ml_dataset_{lead_time}_{target}.feather')
        
        print(np.sum(np.isnan(test_df[f'Baseline__{target}'].values)))
        print(f'Saving {out_file}...')
        test_df.to_feather(out_file)  

/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IsotonicRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_956927/573257866.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Baseline__{target}'] = predictions


0
Saving /work/mflora/ML_DATA/test_data/test_ml_dataset_first_hour_tornado.feather...


/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IsotonicRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_956927/573257866.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Baseline__{target}'] = predictions


0
Saving /work/mflora/ML_DATA/test_data/test_ml_dataset_first_hour_severe_hail.feather...


/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IsotonicRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_956927/573257866.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Baseline__{target}'] = predictions


...

0
Saving /work/mflora/ML_DATA/test_data/test_ml_dataset_second_hour_tornado.feather...


/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IsotonicRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_956927/573257866.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Baseline__{target}'] = predictions


0
Saving /work/mflora/ML_DATA/test_data/test_ml_dataset_second_hour_severe_hail.feather...


/home/monte.flora/miniconda3/envs/wofs_test/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator IsotonicRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_956927/573257866.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Baseline__{target}'] = predictions


0
Saving /work/mflora/ML_DATA/test_data/test_ml_dataset_second_hour_severe_wind.feather...
